<a href="https://colab.research.google.com/github/KyraZzz/partii-work/blob/main/bioinfo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Global alignment

In [1]:
def print_global_align(seq1,seq2,op):
  m = len(seq1); i = m
  n = len(seq2); j = n
  align1 = ""
  align2 = ""

  while i > 0 and j > 0:
    if op[i][j] == 1:
      align1 += seq1[i-1]
      align2 += seq2[j-1]
      i -= 1; j -= 1
    elif op[i][j] == 2:
      align1 += seq1[i-1]
      align2 += "-"
      i -= 1 
    elif op[i][j] == 3:
      align1 += "-"
      align2 += seq2[j-1]
      j -= 1
  
  while i > 0:
    align1 += seq1[i-1]
    align2 += "-"
    i -= 1
  while j > 0:
    align1 += "-"
    align2 += seq2[j-1]
    j -= 1
  
  print(align1[::-1] + "\n" + align2[::-1])

def needleman_wunsch(seq1,seq2,match_award=1,mismatch_pen=-1,indel_pen=-1):
  m = len(seq1)
  n = len(seq2)
  dp = [[0 for _ in range(n+1)] for _ in range(m+1)]
  op = [[0 for _ in range(n+1)] for _ in range(m+1)] # 1: diagonal, 2: right(insert), 3: down(delete)

  # fill in score matrix
  for i in range(1,m+1):
    dp[i][0] = indel_pen * i
  for j in range(1,n+1):
    dp[0][j] = indel_pen * j
  
  for i in range(1,m+1):
    for j in range(1,n+1):
      insert = dp[i-1][j] + indel_pen
      delete = dp[i][j-1] + indel_pen
      match_mismatch = dp[i-1][j-1] + match_award if seq1[i-1] == seq2[j-1] else dp[i-1][j-1] + mismatch_pen
      dp[i][j] = max(insert, delete, match_mismatch)
      if insert == dp[i][j]:
        op[i][j] = 2
      elif delete == dp[i][j]:
        op[i][j] = 3
      else:
        op[i][j] = 1
    
  # print optimal global alignment
  print_global_align(seq1,seq2,op)
  return dp, op

In [2]:
seq1 = "ATTACA"
seq2 = "ATGCT"
needleman_wunsch(seq1,seq2)

ATTACA
ATG-CT


([[0, -1, -2, -3, -4, -5],
  [-1, 1, 0, -1, -2, -3],
  [-2, 0, 2, 1, 0, -1],
  [-3, -1, 1, 1, 0, 1],
  [-4, -2, 0, 0, 0, 0],
  [-5, -3, -1, -1, 1, 0],
  [-6, -4, -2, -2, 0, 0]],
 [[0, 0, 0, 0, 0, 0],
  [0, 1, 3, 3, 3, 3],
  [0, 2, 1, 3, 3, 3],
  [0, 2, 2, 1, 3, 1],
  [0, 2, 2, 2, 1, 2],
  [0, 2, 2, 2, 1, 3],
  [0, 2, 2, 2, 2, 1]])

# Local alignment

In [3]:
import numpy as np
def print_local_align(seq1,seq2,dp,op):
  arr = np.array(dp)
  i,j = np.unravel_index(arr.argmax(),arr.shape)
  align1 = ""
  align2 = ""

  while i > 0 and j > 0 and dp[i][j] != 0:
    if op[i][j] == 1:
      align1 += seq1[i-1]
      align2 += seq2[j-1]
      i -= 1; j -= 1
    elif op[i][j] == 2:
      align1 += seq1[i-1]
      align2 += "-"
      i -= 1 
    elif op[i][j] == 3:
      align1 += "-"
      align2 += seq2[j-1]
      j -= 1
  
  print(align1[::-1] + "\n" + align2[::-1])

def smitch_waterman(seq1,seq2,match_award=1,mismatch_pen=-1,indel_pen=-1):
  m = len(seq1)
  n = len(seq2)
  dp = [[0 for _ in range(n+1)] for _ in range(m+1)]
  op = [[0 for _ in range(n+1)] for _ in range(m+1)] # 1: diagonal, 2: right(insert), 3: down(delete), 4: reset(0)

  # fill in score matrix
  for i in range(1,m+1):
    for j in range(1,n+1):
      insert = dp[i-1][j] + indel_pen
      delete = dp[i][j-1] + indel_pen
      match_mismatch = dp[i-1][j-1] + match_award if seq1[i-1] == seq2[j-1] else dp[i-1][j-1] + mismatch_pen
      dp[i][j] = max(0,insert, delete, match_mismatch)
      if insert == dp[i][j]:
        op[i][j] = 2
      elif delete == dp[i][j]:
        op[i][j] = 3
      elif match_mismatch == dp[i][j]:
        op[i][j] = 1
      else:
        op[i][j] = 4
  
  # print optimal local alignment
  print_local_align(seq1,seq2,dp,op)
  return dp, op

In [4]:
seq1 = "GCCGCCGTCGTTTTCAGCAGTTATGTTCAGAT"
seq2 = "GCCCAGTCTATGTCAGGGGCACGAGCATGCACA"
dp, op = smitch_waterman(seq1,seq2)

CAGT-TATGTTCAG
CAGTCTATGT-CAG


# Alignment with affine gap penalities

In [25]:
def print_global_align_affine_gap_penalty(v,w,F,op):
  m = len(v); i = m
  n = len(w); j = n

  alignv = ""
  alignw = ""

  start_level_choice = [F[0][i][j], F[1][i][j], F[2][i][j]]
  curr_level = start_level_choice.index(max(start_level_choice))

  while i > 0 and j > 0:
    next_level = op[curr_level][i][j] 
    if curr_level == 0:
      alignv += v[i-1]
      alignw += "-"
      i -= 1
    elif next_level == 2:
      alignv += "-"
      alignw += w[j-1]
      j -= 1
    else:
      alignv += v[i-1]
      alignw += w[j-1]
      i -= 1; j -= 1
    curr_level = next_level
  
  while i > 0:
    alignv += v[i-1]
    alignw += "-"
    i -= 1
  while j > 0:
    alignv += "-"
    alignw += w[j-1]
    j -= 1
  
  print(alignv[::-1] + "\n" + alignw[::-1])

def global_alignment_affine_gap_penalty(v,w,scoring_matrix, sigma=2, epsilon=1):
  m = len(v)
  n = len(w)
  F = [[[0 for _ in range(n+1)] for _ in range(m+1)] for _ in range(3)]
  op = [[[0 for _ in range(n+1)] for _ in range(m+1)] for _ in range(3)]

  # initialise 3 level edges
  for i in range(1, m+1):
    F[0][i][0] = -sigma - epsilon * (i-1)
    F[1][i][0] = -sigma - epsilon * (i-1)
    F[2][i][0] = -sigma - epsilon * (i-1)
  for j in range(1, n+1):
    F[0][0][j] = -sigma - epsilon * (j-1)
    F[1][0][j] = -sigma - epsilon * (j-1)
    F[2][0][j] = -sigma - epsilon * (j-1)

  # filling F and op for 3 levels
  for i in range(1,m+1):
    for j in range(1,n+1):
      lower_scores = [F[0][i-1][j] - epsilon, F[1][i-1][j] - sigma]
      F[0][i][j] = max(lower_scores)
      op[0][i][j] = lower_scores.index(F[0][i][j])
      upper_scores = [F[1][i][j-1] - sigma,F[2][i][j-1] - epsilon]
      F[2][i][j] = max(upper_scores)
      op[2][i][j] = upper_scores.index(F[2][i][j]) + 1
      middle_scores = [F[0][i][j],F[1][i-1][j-1]+scoring_matrix[i][j],F[2][i][j]]
      F[1][i][j] = max(middle_scores)
      op[1][i][j] = middle_scores.index(F[1][i][j])
    
  # print optimal global alignment
  print_global_align_affine_gap_penalty(v,w,F,op)
  return F, op

In [26]:
seq1 = "GATCCAG"
seq2 = "GACAG"
scoring_matrix = [[-1 for _ in range(len(seq2)+1)] for _ in range(len(seq1)+1)]
for i in range(1,len(seq1)+1):
  for j in range(1,len(seq2)+1):
    if seq1[i-1] == seq2[j-1]:
      scoring_matrix[i][j] = 1
    else:
      scoring_matrix[i][j] = -1
print(scoring_matrix)
global_alignment_affine_gap_penalty(seq1,seq2,scoring_matrix)

[[-1, -1, -1, -1, -1, -1], [-1, 1, -1, -1, -1, 1], [-1, -1, 1, -1, 1, -1], [-1, -1, -1, -1, -1, -1], [-1, -1, -1, 1, -1, -1], [-1, -1, -1, 1, -1, -1], [-1, -1, 1, -1, 1, -1], [-1, 1, -1, -1, -1, 1]]
GATCCAG
G--ACAG


([[[0, -2, -3, -4, -5, -6],
   [-2, -3, -4, -5, -6, -7],
   [-3, -1, -3, -4, -5, -6],
   [-4, -2, 0, -2, -3, -4],
   [-5, -3, -1, -1, -3, -4],
   [-6, -4, -2, -1, -2, -4],
   [-7, -5, -3, -2, -2, -3],
   [-8, -6, -4, -3, -1, -3]],
  [[0, -2, -3, -4, -5, -6],
   [-2, 1, -1, -2, -3, -4],
   [-3, -1, 2, 0, -1, -2],
   [-4, -2, 0, 1, -1, -2],
   [-5, -3, -1, 1, 0, -2],
   [-6, -4, -2, 0, 0, -1],
   [-7, -5, -3, -2, 1, -1],
   [-8, -6, -4, -3, -1, 2]],
  [[0, -2, -3, -4, -5, -6],
   [-2, -3, -1, -2, -3, -4],
   [-3, -4, -3, 0, -1, -2],
   [-4, -5, -4, -2, -1, -2],
   [-5, -6, -5, -3, -1, -2],
   [-6, -7, -6, -4, -2, -2],
   [-7, -8, -7, -5, -4, -1],
   [-8, -9, -8, -6, -5, -3]]],
 [[[0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0],
   [0, 1, 1, 1, 1, 1],
   [0, 0, 1, 1, 1, 1],
   [0, 0, 0, 1, 1, 1],
   [0, 0, 0, 1, 1, 1],
   [0, 0, 0, 0, 1, 1],
   [0, 0, 0, 0, 1, 1]],
  [[0, 0, 0, 0, 0, 0],
   [0, 1, 2, 2, 2, 1],
   [0, 0, 1, 2, 1, 2],
   [0, 0, 0, 1, 1, 1],
   [0, 0, 0, 1, 1, 1],
   [0, 0, 0, 1, 